In [1]:
import numpy as np
import pandas as pd

#CentrinManualScore CSV location of intephase labels
fp = r'\\allen\aics\assay-dev\MicroscopyOtherData\Ruian\Projects\aicsassaydevknowledgetransfer\interphase_label\3500001603_all_cell_scores_mitosislabel_interphaselabel.csv'

df = pd.read_csv(fp)
print(df.shape)
df.head(2)

(610, 30)


,Index,Version,inputFolder,inputFilename,imageXyPixelSize,imageZPixelSize,plate_ID,well_ID,position_ID,colony_position,...,outputThisCellIndex,Labeler1,Score1,Labeler2,Score2,FinalScore,MitosisLabel,InterphaseLabel,GoodNucBadCell,Unnamed: 29
0,1,3,//allen/aics/microscopy/PRODUCTION/PIPELINE_4_...,3500001603_100X_20171222_1-Scene-01-P1-E04.czi,0.108,0.29,3500001603,E04,P01,NaN,...,1,Ruian,-1,Sara,-1,-1,5,Nan,0,NaN
1,2,3,//allen/aics/microscopy/PRODUCTION/PIPELINE_4_...,3500001603_100X_20171222_1-Scene-01-P1-E04.czi,0.108,0.29,3500001603,E04,P01,NaN,...,2,Ruian,1,Sara,1,1,7,Nan,0,NaN


In [2]:
# QUILT MANIFEST 
fp1 = r"\\allen\aics\assay-dev\users\Frick\PythonProjects\WHEREdoes_this_go\metadata.csv"
df1 = pd.read_csv(fp1)

In [3]:
#refine quilt manifest to centrin, and correct plate id and the well_ids from CentrinManualScore csv

#first get the list of wells and list of plate ids from CentrinManualScore CSV
well_list = df['well_ID'].unique()
plate_list = df['plate_ID'].unique()
print(well_list)
print(plate_list)

dfc = df1[df1['Protein'].str.contains('Centrin')] #choose centrin
well_log = np.asarray([True if x[0]+'0'+x[1] in well_list else False for x in dfc['WellName'].tolist()])
well_log = np.asarray([True if x in plate_list else False for x in dfc['PlateId'].tolist()])
dfcp = dfc[well_log]
dfcp.head(1)

['E04' 'E06' 'E07' 'E08' 'F04' 'F05' 'F06' 'F07' 'F08']
[3500001603]


,CellId,CellIndex,CellLine,CellLineId,CellLineId/Name,CellPopulationId,ChannelNumber405,ChannelNumber638,ChannelNumberBrightfield,ChannelNumberStruct,...,ch_memb,ch_seg_cell,ch_seg_nuc,ch_struct,ch_trans,save_feats_path,save_reg_path,save_reg_path_flat,save_reg_path_flat_proj,FeatureExplorerURL
12902,62651,1,AICS-32,35,AICS-32,5379,5,1,6,3,...,3,1,0,4,5,cell_features/7b8b5c74_9938_62651_feats.json,cell_images_3d/5a810e7c_9938_62651_reg.tiff,cell_images_2d/c58fdb7d_9938_62651_reg_flat.png,cell_images_2d_projections/94e912f9_9938_62651...,https://cfe.allencell.org/?cellSelectedFor3D=6...


In [12]:
#now check per well
#CentrinManualScore csv
dfkeep = df[df['FinalScore']==1]
for plate,group in dfkeep.groupby('plate_ID'):
    for well,g in group.groupby('well_ID'):
        dfg = g.groupby(['position_ID']).first()
        print(well,"number of FOVS",dfg.shape[0],'//',"number of cells",g.shape[0])
#         for fov,gg in g.groupby(['position_ID']):
#             print(gg['outputThisCellIndex'].to_list())

E04 number of FOVS 5 // number of cells 23
E06 number of FOVS 6 // number of cells 28
E07 number of FOVS 7 // number of cells 19
E08 number of FOVS 4 // number of cells 13
F04 number of FOVS 7 // number of cells 47
F05 number of FOVS 8 // number of cells 32
F06 number of FOVS 3 // number of cells 11
F07 number of FOVS 5 // number of cells 18
F08 number of FOVS 1 // number of cells 3


In [20]:
#manifest csv
for plate,group in dfcp.groupby('PlateId'):
    for well,g in group.groupby('WellName'):
        dfg = g.groupby(['FOVId']).first()
        print(well,"number of FOVS",dfg.shape[0],'//',"number of cells",g.shape[0])
#         for fov,gg in g.groupby(['FOVId']):
#             print(gg['CellIndex'].to_list())
       

E4 number of FOVS 5 // number of cells 26
E6 number of FOVS 6 // number of cells 25
E7 number of FOVS 6 // number of cells 22
E8 number of FOVS 4 // number of cells 16
F4 number of FOVS 7 // number of cells 44
F5 number of FOVS 8 // number of cells 23
F6 number of FOVS 3 // number of cells 7
F7 number of FOVS 5 // number of cells 15
F8 number of FOVS 1 // number of cells 2


In [37]:
#now check per well
#CentrinManualScore csv
tcau = 0
dfkeep = df[df['FinalScore']==1]
for plate,group in dfkeep.groupby('plate_ID'):
    for well,g in group.groupby('well_ID'):
        dfg = g.groupby(['position_ID']).first()
        for fov,gg in g.groupby(['position_ID']):
            print(well,fov,gg['outputThisCellIndex'].to_list())
            tcau+=gg['outputThisCellIndex'].unique().shape[0]
        print('\n')
print("total cells after unique",tcau)

E04 P01 [2, 5]
E04 P02 [1, 3, 5]
E04 P04 [3, 4, 5, 10, 11, 2, 8, 13]
E04 P05 [2, 5, 6, 8, 10, 11, 9]
E04 P06 [6, 9, 11]


E06 P07 [1, 2]
E06 P09 [4, 5, 9, 2]
E06 P11 [1, 3, 6]
E06 P12 [2, 8, 1, 3, 5, 6]
E06 P13 [2, 9, 12, 8, 13]
E06 P14 [1, 6, 7, 11, 3, 4, 8, 12]


E07 P15 [1, 2, 4, 10]
E07 P16 [4]
E07 P17 [1, 3, 4, 7, 8]
E07 P18 [11]
E07 P20 [2, 3, 7]
E07 P21 [1, 5, 6]
E07 P22 [2, 12]


E08 P24 [6, 9]
E08 P25 [5, 6, 8, 9, 1]
E08 P26 [4, 5, 2]
E08 P28 [4, 8, 3]


F04 P29 [1, 8, 9, 10, 6, 11]
F04 P30 [1, 3]
F04 P32 [1, 3, 4, 9, 6, 7, 8, 15]
F04 P33 [3, 4, 7, 9, 1, 2, 11, 12, 13, 14]
F04 P34 [2, 3, 6, 7, 11]
F04 P35 [1, 3, 5, 9, 11, 12, 2, 4, 13, 16]
F04 P38 [1, 4, 8, 15, 10, 11]


F05 P40 [5, 8, 1, 7, 11]
F05 P41 [6, 2, 5, 12]
F05 P42 [6, 7]
F05 P44 [1, 3, 5]
F05 P45 [1, 2, 4, 8, 3, 6]
F05 P46 [1, 2, 9, 11]
F05 P47 [1, 2, 5, 3, 4, 8]
F05 P49 [1, 9]


F06 P52 [1, 8, 9, 10, 6]
F06 P54 [2, 4, 8, 5]
F06 P55 [5, 6]


F07 P56 [1, 7, 8, 9, 10, 3]
F07 P58 [5, 8]
F07 P59 [8, 10, 11]
F07 P60 [4, 

In [36]:
#manifest csv
tcau = 0 #total cells after unique
for plate,group in dfcp.groupby('PlateId'):
    for well,g in group.groupby('WellName'):
        dfg = g.groupby(['FOVId']).first()
        for fov,gg in g.groupby(['FOVId']):
#             print(well,fov,gg['CellIndex'].to_list())
            print(well,fov,gg['CellIndex'].unique())
            #now since some cell indexes appear repeated for some reason...
            #see how many unique cell indexes there are
            tcau += gg['CellIndex'].unique().shape[0]
        print('\n')
print("total cells after unique",tcau)

E4 9920 [ 2  4  5  6  8  9 10]
E4 9938 [1 3 5]
E4 9955 [5 9]
E4 9966 [6 7]
E4 9972 [ 3  4  5  9 11 13 14]


E6 9911 [2 4 5 6 9]
E6 9921 [1 2 3 5 8]
E6 9923 [ 1 10]
E6 9929 [1 3 6]
E6 9944 [2 8 9]
E6 9965 [ 1  4  5  7  8 11 12]


E7 9919 [1 5 6]
E7 9922 [2 3 7]
E7 9924 [1 3 7 8]
E7 9945 [11 12]
E7 9946 [2 5]
E7 9949 [ 1  9 12]


E8 9915 [2 6 9]
E8 9925 [1 5 6 9]
E8 9974 [3 4]
E8 9978 [2 5]


F4 9913 [1 3 4 6 7 8]
F4 9918 [ 2  3  4  9 12 13 16 17]
F4 9928 [ 1  4  6  8  9 10 15]
F4 9930 [ 1  6  8 11]
F4 9934 [3 6]
F4 9959 [1 3]
F4 9964 [ 1  2  3  4  7  9 11 12 13 14]


F5 9917 [10]
F5 9926 [6]
F5 9931 [1 5]
F5 9941 [1 3 5]
F5 9947 [1 4 5 8]
F5 9950 [2 5]
F5 9962 [1 2 3 4 5 6 8]
F5 9971 [1 5 7]


F6 9936 [ 1  9 10]
F6 9961 [5 6]
F6 9975 [2 4]


F7 9932 [3 8]
F7 9952 [7]
F7 9954 [3 5 8]
F7 9958 [ 7 13]
F7 9967 [1 8]


F8 9977 [1 2]


total cells after unique 155


In [29]:
dfcp.columns

Index(['CellId', 'CellIndex', 'CellLine', 'CellLineId', 'CellLineId/Name',
       'CellPopulationId', 'ChannelNumber405', 'ChannelNumber638',
       'ChannelNumberBrightfield', 'ChannelNumberStruct', 'Clone', 'Col',
       'DataSetId', 'FOVId', 'Gene', 'InstrumentId', 'MembraneContourFileId',
       'MembraneSegmentationFileId', 'NucMembSegmentationAlgorithm',
       'NucMembSegmentationAlgorithmVersion', 'NucleusContourFileId',
       'NucleusSegmentationFileId', 'Objective', 'Passage', 'PixelScaleX',
       'PixelScaleY', 'PixelScaleZ', 'PlateId', 'Protein',
       'ProteinDisplayName', 'ProteinId/Name', 'Row', 'SourceFileId',
       'Structure', 'StructureContourFileId', 'StructureDisplayName',
       'StructureId', 'StructureId/Name', 'StructureSegmentationAlgorithm',
       'StructureSegmentationAlgorithmVersion', 'StructureSegmentationFileId',
       'WellId', 'WellName', 'Workflow', 'WorkflowId', 'ch_dna', 'ch_memb',
       'ch_seg_cell', 'ch_seg_nuc', 'ch_struct', 'ch_trans', '

In [32]:
dfcp.SourceFileId

12902    4dd5f74aaa3b471ebed274b034b59068
12903    4dd5f74aaa3b471ebed274b034b59068
12904    4dd5f74aaa3b471ebed274b034b59068
12905    18c46fbfef43439cb7e8078146869899
12906    18c46fbfef43439cb7e8078146869899
                       ...               
42515    40840faefc434ce283dd96435f3fafe0
42516    40840faefc434ce283dd96435f3fafe0
42517    40840faefc434ce283dd96435f3fafe0
42518    40840faefc434ce283dd96435f3fafe0
42519    40840faefc434ce283dd96435f3fafe0
Name: SourceFileId, Length: 180, dtype: object